In [1]:
!pip install pandas pyarrow yfinance fortitudo_tech matplotlib

  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached fortitudo_tech-1.1.12-py3-none-any.whl.metadata (5.8 kB)
  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached frozendict-2.4.7-py3-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.3-py3-none-any.whl
  Using cached curl_cffi-0.13.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached protobuf-6.33.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.ma

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
import fortitudo.tech as ft
from cvxopt import matrix

# --- 1. DATA LOADING & ALIGNMENT ---
print("Loading and aligning data...")
data = pd.read_parquet("merged_portfolio_data.parquet")
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')

# Drop NA columns
data_clean = data.dropna(axis=1, how='any')

# Download VIX
start_date = data_clean.index[0].strftime('%Y-%m-%d')
end_date = data_clean.index[-1].strftime('%Y-%m-%d')
vix_raw = yf.download('^VIX', start=start_date, end=end_date, progress=False)
if isinstance(vix_raw.columns, pd.MultiIndex):
    vix_raw = vix_raw['Close']['^VIX']
else:
    vix_raw = vix_raw['Close']

# Align timestamps
vix_aligned = vix_raw.reindex(data_clean.index).ffill().dropna()
common_dates = data_clean.index.intersection(vix_aligned.index)
asset_data = data_clean.loc[common_dates]
vix_aligned = vix_aligned.loc[common_dates]

# Calculate Returns
pnl = np.log(asset_data / asset_data.shift(1)).dropna()
vix_aligned = vix_aligned.loc[pnl.index]

print(f"Data aligned: {pnl.shape[1]} assets, {pnl.shape[0]} days")

# --- 2. FULL UNIVERSE CONSTRAINTS ---
# We need these matrices to run the selection loop on all 576 assets
I_full = pnl.shape[1]
R_full = pnl.values

# Constraints: Long Only (0.0) to Cap (0.25)
upper_bound = 0.25
lower_bound = 0.0

# Geometric Constraints (Gx <= h)
G_full = np.vstack((np.eye(I_full), -np.eye(I_full)))
h_full = np.hstack((upper_bound * np.ones(I_full), -lower_bound * np.ones(I_full)))

# Budget Constraint (Ax = b implies sum(w)=1)
v_full = np.ones(I_full)

print("Full universe constraints ready.")

Loading and aligning data...
Data aligned: 576 assets, 2468 days
Full universe constraints ready.


/tmp/ipykernel_723/2048252361.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix_raw = yf.download('^VIX', start=start_date, end=end_date, progress=False)
/usr/local/lib/python3.12/dist-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


In [9]:
# --- 3. ROBUST STABILITY SELECTION ---
print("\n=== ROBUST PRE-SELECTION (BOOTSTRAP) ===")

# Parameters for Selection
B_select = 30      # Number of bootstraps (30 is enough for selection, 576 assets is heavy)
N_select = 100     # Length of each bootstrapped history
alphas = [0.95]    # We only need one alpha here to find "generally good" assets
P_points = 3       # Points on the frontier per run (Low, Med, High risk)

# Storage for votes
# We will simply sum up the weights assigned to assets across all runs
accumulated_weights = np.zeros(I_full)

np.random.seed(42) # Reproducibility

print(f"Running {B_select} bootstraps on {I_full} assets...")

for b in range(B_select):
    # A. Bootstrap Resampling (Standard Uniform, No VIX views)
    # We pick N_select random days from the history with replacement
    idx = np.random.choice(np.arange(len(R_full)), size=N_select, replace=True)
    R_boot = R_full[idx]
    
    # B. Optimize on this "Alternative Reality"
    # We use the MeanCVaR optimizer directly on the raw bootstrapped data
    try:
        for alpha in alphas:
            # Initialize optimizer
            # Note: We pass 'v_full' to enforce sum(w)=1 internally if needed by the helper
            optimizer = ft.MeanCVaR(R_boot, G_full, h_full, v=v_full, alpha=alpha)
            
            # Get efficient frontier (weights)
            frontier_weights = optimizer.efficient_frontier(P_points)
            
            # C. Accumulate "Votes"
            # We add the absolute weights. If an asset is chosen often, this sum grows large.
            accumulated_weights += np.sum(np.abs(frontier_weights), axis=1)
            
    except Exception as e:
        # Solvers sometimes fail on singular matrices in small bootstraps
        continue

    if (b+1) % 5 == 0:
        print(f"  Completed {b+1}/{B_select} runs...")

print("Robust selection complete.")


=== ROBUST PRE-SELECTION (BOOTSTRAP) ===
Running 30 bootstraps on 576 assets...
  Completed 5/30 runs...
  Completed 10/30 runs...
  Completed 15/30 runs...
  Completed 20/30 runs...
  Completed 25/30 runs...
  Completed 30/30 runs...
Robust selection complete.


In [12]:
# --- 4. SELECT TOP K & RESIZE ---
K = 20
# Get indices of the heaviest weights
top_indices = np.argsort(accumulated_weights)[-K:]
top_indices = np.sort(top_indices) # Keep original column order

# Subset the Data
R = R_full[:, top_indices]
asset_names = pnl.columns[top_indices]
print(f"\nSelected Top {K} Assets (Stable across bootstraps):")
print(asset_names.values[:20])

# --- CRITICAL: UPDATE CONSTRAINTS FOR SUBSET ---
I = K
v = np.ones(I)

# Resize G and h for 20 assets (instead of 576)
G = np.vstack((np.eye(I), -np.eye(I)))
h = np.hstack((upper_bound * np.ones(I), -lower_bound * np.ones(I)))

# Update Prior Stats for later use
prior_stats = ft.simulation_moments(pnl.iloc[:, top_indices])

print(f"Constraints resized. Ready for VIX Regime Optimization.")


Selected Top 20 Assets (Stable across bootstraps):
['BTU' 'CA' 'EMC' 'ENPH' 'EXE' 'GME' 'GRN' 'KG' 'KR' 'NVDA' 'SE' 'SHLD'
 'SMCI' 'GC=F' 'NG=F' 'EUR=X' 'JPY=X' 'GBP=X' 'AUD=X' 'CHF=X']
Constraints resized. Ready for VIX Regime Optimization.


In [13]:
# --- 5. VIX REGIME SETUP & OPTIMIZATION ---
print("\n=== STAGE 2: VIX REGIME OPTIMIZATION ===")

# A. Define VIX States
vix_33 = vix_aligned.quantile(0.33)
vix_66 = vix_aligned.quantile(0.66)
vix_state = np.zeros(len(vix_aligned))
vix_state[(vix_aligned > vix_33) & (vix_aligned <= vix_66)] = 1
vix_state[vix_aligned > vix_66] = 2

current_vix = vix_aligned.iloc[-1]
if current_vix <= vix_33: current_state = 0; state_name = 'LOW'
elif current_vix <= vix_66: current_state = 1; state_name = 'MEDIUM'
else: current_state = 2; state_name = 'HIGH'

print(f"Current VIX: {current_vix:.2f} -> State: {state_name}")

# B. Time Decay & Conditioning
T_tilde = len(pnl)
p_exp = ft.exp_decay_probs(pnl, half_life=T_tilde/2)

# Condition on State (Entropy Pooling Step 1)
A_state = vix_state[np.newaxis, :]
b_state = np.array([[current_state]])
q_state = ft.entropy_pooling(p_exp, A=A_state, b=b_state)

# C. Regime-Conditional Bootstrap Loop
B_final = 99   # High number for smooth results
P_final = 9    # Granular frontier
alphas_final = [0.9, 0.95, 0.975]

# Storage
results = np.full((len(v), P_final, 3 * B_final), np.nan)

# Bootstrapping from the Regime-Specific Mean/Cov
R_regime = R[vix_state == current_state]
means_regime = R_regime.mean(axis=0)
cov_regime = np.cov(R_regime, rowvar=False)

# Generate scenarios (N=100 observations per bootstrap)
return_sim = np.random.multivariate_normal(means_regime, cov_regime, (100, B_final))

print(f"Running {B_final} regime-conditioned bootstraps...")
counter = 0

for b in range(B_final):
    # View on Expected Returns (Uncertainty)
    means_uncertainty = np.mean(return_sim[:, b, :], axis=0)
    
    # Entropy Pooling Step 2: Impose the uncertain mean as a view
    # Note: We use R (20 assets) here
    q = ft.entropy_pooling(q_state, A=R.T, b=means_uncertainty[:, np.newaxis])
    
    # Optimization Loop
    for alpha in alphas_final:
        # Calculate implied returns for the optimizer
        means_run = q.T @ R
        
        # Solve
        cvar_opt = ft.MeanCVaR(R, G, h, v=v, alpha=alpha, p=q)
        # Force the expected return view into the solver structure
        cvar_opt._expected_return_row = -matrix(np.hstack((means_run, np.zeros((1, 2)))))
        
        # Store Frontier
        frontier = cvar_opt.efficient_frontier(P_final)
        results[:, :, counter] = frontier
        counter += 1

print(f"Optimization Complete. Generated {counter} portfolios.")


=== STAGE 2: VIX REGIME OPTIMIZATION ===
Current VIX: 17.60 -> State: MEDIUM
Running 99 regime-conditioned bootstraps...
Optimization Complete. Generated 297 portfolios.


In [17]:
# --- 6. DETAILED STACKING & METRICS ---
print("\n=== FINAL ANALYSIS: STACKING BY ALPHA ===")

pf_index = 4  # Middle of the risk curve
L = 15        # Number of folds for stacking
floor = 0.00  # Set to 0.00 to see all theoretical weights

# 1. Separate the results tensor by Alpha
# We filled the tensor in the order: [b0_a90, b0_a95, b0_a975, b1_a90...]
# So we slice with a step of 3 to grab every 3rd result.
results_90 = results[:, pf_index, 0::3]
results_95 = results[:, pf_index, 1::3]
results_975 = results[:, pf_index, 2::3]
results_combined = results[:, pf_index, :] # All mixed

# 2. Run Exposure Stacking on each slice
w_90 = ft.exposure_stacking(L, results_90)
w_95 = ft.exposure_stacking(L, results_95)
w_975 = ft.exposure_stacking(L, results_975)
w_comb = ft.exposure_stacking(L, results_combined)

# Helper to clean weights
def clean_w(w, f):
    w[np.abs(w) < f] = 0
    return w / np.sum(w)

weights_dict = {
    'CVaR 90%': clean_w(w_90, floor),
    'CVaR 95%': clean_w(w_95, floor),
    'CVaR 97.5%': clean_w(w_975, floor),
    'Combined': clean_w(w_comb, floor)
}

# 3. Create DataFrame
df_final = pd.DataFrame(weights_dict, index=asset_names)

# 4. Calculate Portfolio Metrics
# We use the subset 'R' (20 assets) and 'prior_stats' for calculation
stats_rows = {}

for name, w in weights_dict.items():
    # Calculate portfolio return series (Log returns)
    pf_log_rets = R @ w
    
    # Convert to Simple Returns for accurate real-world stats
    pf_simple_rets = np.exp(pf_log_rets) - 1
    
    # Metrics
    mu = np.mean(pf_simple_rets) * 252         # Annualized Mean
    vol = np.std(pf_simple_rets) * np.sqrt(252) # Annualized Vol
    sharpe = mu / vol if vol > 0 else 0
    
    # Historical CVaR (95%) of this specific portfolio configuration
    # We sort returns and take the average of the worst 5%
    sorted_rets = np.sort(pf_simple_rets)
    cutoff = int(0.05 * len(sorted_rets))
    cvar_hist = np.mean(sorted_rets[:cutoff]) * np.sqrt(252) # Annualized Scale
    
    stats_rows[name] = {
        'Ann. Return': mu,
        'Ann. Volatility': vol,
        'Sharpe Ratio': sharpe,
        'Hist. CVaR (95%)': cvar_hist
    }

# 5. Append Metrics to DataFrame
stats_df = pd.DataFrame(stats_rows)
df_display = pd.concat([df_final * 100, stats_df]) # Weights in %

# Formatting
print(f"\nFinal Allocations (Floor: {floor*100}%)")
print("-" * 60)
# Show only assets with at least one non-zero weight in the 'Combined' column
active_assets = df_display.index[df_display['Combined'].abs() > 0.001]
# Ensure stats rows are included
final_idx = list(active_assets) + list(stats_df.index)
# Remove duplicates if any and preserve order
final_idx = sorted(set(final_idx), key=lambda x: list(df_display.index).index(x))

print(df_display.loc[final_idx].round(4))


=== FINAL ANALYSIS: STACKING BY ALPHA ===

Final Allocations (Floor: 0.0%)
------------------------------------------------------------
                  CVaR 90%  CVaR 95%  CVaR 97.5%  Combined
BTU                 0.5742    0.7031      0.9118    0.8705
CA                  7.2714    7.2833      6.7124    6.6752
EMC                 5.0360    5.0612      5.2705    5.1100
ENPH                1.3525    1.3219      1.2335    1.5491
EXE                 2.2994    2.3536      2.1626    2.1687
GME                 0.3017    0.3306      0.3310    0.4545
GRN                 7.8093    7.1028      6.5146    7.4637
KG                  3.3527    3.6764      4.0616    3.4055
KR                  7.4083    7.5192      7.4667    7.7678
NVDA                7.3798    7.3100      7.4273    7.4354
SE                  2.4188    2.2600      2.3775    2.4348
SHLD                4.4016    4.4419      4.4347    4.2871
SMCI                3.2471    3.3438      3.6815    3.2371
GC=F               12.5072   12.3043 